In [1]:
import importlib
import visualisation
import numpy as np
import LinearRegression
from visualisation import plot_kernel_density, plot_multiline_chart
importlib.reload(visualisation)
importlib.reload(LinearRegression)
import torch

In [6]:
from utilities import TimeSeries
import polars as pl 
SHIT_COINS = ['DOGE-USD','SHIB-USD']
DECENT_COINS = ['BTC-USD','ADA-USD']
STABLE_COINS = ['USDT-USD']

TICKERS = SHIT_COINS + DECENT_COINS + STABLE_COINS
def get_market_data (ticker):
    ticker = TimeSeries(ticker, "2022-01-01").construct_returns()
    return pl.DataFrame(ticker.data).with_columns([pl.col("Date").cast(pl.Date)])

def get_return_vector (ticker_df):
    return ticker_df.select("Returns").drop_nulls().to_numpy().flatten()

# return_vectors = [(get_return_vector(get_market_data(ticker)), ticker) for ticker in TICKERS]
return_vectors = [get_market_data(ticker).write_csv(f"C:/git/WQU/PortfolioManagement/{ticker}.csv") for ticker in TICKERS]


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [ ]:
plot_kernel_density([return_vectors[2],return_vectors[3]],labels=True, title="KDE of Decent Coin Returns", x_label="Returns", y_label="Density")

In [ ]:
plot_kernel_density([return_vectors[0],return_vectors[1]],labels=True, title="KDE of Shitcoin Returns", x_label="Returns", y_label="Density")

In [ ]:
plot_kernel_density([return_vectors[4]],labels=True, title="KDE of Stablecoin Returns", x_label="Returns", y_label="Density")

In [ ]:
def generate_random_portfolio_weights(size, number_of_assets):
    shape = [size, number_of_assets]
    random_tensor = torch.rand(shape) * 0.5
    return random_tensor / torch.sum(random_tensor, dim=1, keepdim=True)

def get_covariance_matrix(data):
    mean = torch.mean(data, dim=0)
    centered_data = data - mean
    cov_matrix = torch.matmul(centered_data.t(), centered_data) / data.shape[0]
    return cov_matrix

In [ ]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')

def create_tensors (device, asset_returns):
    if device == torch.device('cuda'):
        # print("USING GPU")
        # print("Can use a larger number of allocations")
        number_of_random_allocations = 50000
    else:
        # print("USING CPU")
        # print("Keep number of allocations below 10000")
        number_of_random_allocations = 20000
    asset_returns =  asset_returns.T.to(device)
    allocation = generate_random_portfolio_weights(number_of_random_allocations, 5).T.to(device)
    return asset_returns, allocation





In [ ]:
return_tensors = [torch.Tensor(r[0]) for r in return_vectors]
asset_returns_stacked = torch.stack((return_tensors))
asset_returns, allocation = create_tensors(gpu, asset_returns=asset_returns_stacked)
covariance_matrix_asset_return = get_covariance_matrix(asset_returns)
portfolio_variance = torch.diag(allocation.T @ covariance_matrix_asset_return @ allocation)
average_returns = torch.mean(asset_returns, dim=0).reshape(1, -1)
mean_vector = average_returns @ allocation
stddev_vector = torch.sqrt(portfolio_variance)

In [ ]:
from visualisation import plot_multiscatter
plot_multiscatter([(stddev_vector.cpu().numpy(),mean_vector.cpu().numpy() ,"Risk vs Return")], title="Random Asset Allocations",x_label="Risk", y_label="Return", markersize=10)


In [ ]:
import pickle
from scipy import optimize
import cvxopt as opt
from cvxopt import blas, solvers

n = len(asset_returns.T)
returns = np.asmatrix(asset_returns.T.cpu().numpy().astype(np.double))    
N = 100000
mus = [100**(5.0 * t/N - 1.0) for t in range(N)]
S = opt.matrix(covariance_matrix_asset_return.cpu().numpy().astype(np.double))
pbar = opt.matrix(np.mean(returns, axis=1))


non_negativity_constraint_matrix = -opt.matrix(np.eye(n))  
zero_vector = opt.matrix(0.0, (n ,1))
full_investment_constraint_matrix = opt.matrix(1.0, (1, n))
ones_vector = opt.matrix(1.0)

# Note: Please do not run the quadratic programming solution in this notebook as it may take a significant amount of time to execute. The optimal portfolio has already been calculated and saved as a pickle file.

# portfolios = [solvers.qp(mu*S, -pbar, non_negativity_constraint_matrix, zero_vector, full_investment_constraint_matrix, ones_vector)['x'] for mu in mus]

with open("optimal_portfolio.pkl", 'rb') as handle:
    portfolios = pickle.load(handle)
returns = torch.Tensor([blas.dot(pbar, x) for x in portfolios])
risks = torch.Tensor([np.sqrt(blas.dot(x, S*x)) for x in portfolios])

In [ ]:
def find_portfolio_with_maximum_sharpe_ratio (risks, returns):
    sharpe = returns / risks
    index = torch.argmax(sharpe)
    return risks[index].item(), returns[index].item()

best_portfolio_std, best_portfolio_mean = find_portfolio_with_maximum_sharpe_ratio(risks, returns)
eqn_of_the_line = lambda x : ( (best_portfolio_mean-risk_free_rate) / best_portfolio_std ) * x + risk_free_rate  
cml_domain = np.linspace(0,0.06,10000)
cml_range = np.array([eqn_of_the_line(i) for i in cml_domain])
from visualisation import plot_multiscatter

plot_multiscatter(
    [(stddev_vector.cpu().numpy(), mean_vector.cpu().numpy(), "Random Portfolio",'o',5,0.5),
     (risks, returns, "Efficient Frontier", '_'),
     (cml_domain, cml_range, "Capital Market Line", "_", 5, 0.5),
     (best_portfolio_std, best_portfolio_mean, "Optimal Portfolio", 'D', 50, 1)],
    title="Optimal Portfolio",
    x_label="Risk",
    y_label="Return",
    markersize=5,
)


In [4]:
import statsmodels.api as sm
import polars as pl
import datetime
df = pl.read_csv("FF3.CSV").with_columns(pl.col("").cast(pl.Utf8).str.to_date(format="%Y%m%d").alias("Date")).filter(pl.col("Date").is_in(pl.Series(gme['Date']).cast(pl.Date))).drop("").join(gme.select(pl.col("Date"), pl.col("Returns")), on="Date").with_columns((pl.col("Returns")-pl.col("RF")).alias("ExcessReturns")).drop_nulls()
x=df.select(pl.col('SMB'), pl.col('HML'), pl.col('Mkt-RF')).to_numpy()
y=df.select(pl.col('ExcessReturns')).to_numpy()
model = sm.OLS(y, sm.add_constant(x))
model.fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     36.62
Date:                Thu, 02 May 2024   Prob (F-statistic):           1.67e-22
Time:                        22:05:28   Log-Likelihood:                 974.11
No. Observations:                1046   AIC:                            -1940.
Df Residuals:                    1042   BIC:                            -1920.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0012      0.003     -0.422      0.673      -0.007       0.005
x1             0.0322      0.004      8.366      0.000       0.025       0.040
x2             0.0041      0.002      1.686      0.092      -0.001       0.009
x3             0.0083      0.002      4.001      0.000       0.004       0.012
==============================================================================
Omnibus:                     1173.597   Durbin-Watson:                   1.782
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           166310.298
Skew:                           5.253   Prob(JB):                         0.00
Kurtosis:                      63.873   Cond. No.                         1.95
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
import LinearRegression
import statsmodels.api as sm
import polars as pl
import importlib
importlib.reload(LinearRegression)
from LinearRegression import RegressionUtility, RegressionType
regression = RegressionUtility(device='cuda', x=x, y=y, regression_type=RegressionType.Linear, epochs=50000, delta_to_early_stop=1e-20, learning_rate=0.01, variable_names=['SMB', 'HML', 'Mkt-RF'])
regression.fit()
print(regression)

In [ ]:
regression.fit()

In [ ]:
print(regression)

In [ ]:
from visualisation import plot_kernel_density
res = plot_kernel_density([(regression.residuals.detach().cpu().numpy(), "Residuals")], title="Residuals")

In [ ]:

importlib.reload(visualisation)
from visualisation import plot_multiscatter
plot = plot_multiscatter([(regression.y_pred_numpy, regression.residuals.detach().cpu().numpy(), "Residuals vs Fitted")], title="Residuals vs Fitted", x_label="Fitted", y_label="Residuals", markersize=20)

In [ ]:
3//2


In [ ]:
from visualisation import plot_multiscatter

plot_multiscatter(
    [(stddev_vector.cpu().numpy(), mean_vector.cpu().numpy(), "Risk vs Return"),
     (risks, returns, "Efficient Frontier")],
    title="Efficient Frontier",
    x_label="Risk",
    y_label="Return",
    markersize=5,
)
